In [ ]:
! pip install semantic-kernel -U
! pip install qdrant_client -U

In [ ]:
import os
import json

In [ ]:
import semantic_kernel as sk
from dotenv import load_dotenv
import semantic_kernel.connectors.ai.open_ai as skaoai

In [ ]:
kernel = sk.Kernel()
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

In [ ]:
kernel.add_chat_service("azure_chat_competion_service", skaoai.AzureChatCompletion("gpt-35-turbo",endpoint,api_key=api_key,api_version = "2023-07-01-preview"))


kernel.add_text_embedding_generation_service(
        "embeddings_services", skaoai.AzureTextEmbedding("text-embedding-ada-002", endpoint,api_key=api_key,api_version = "2023-07-01-preview")
)

In [ ]:
base_plugin = "./plugins"

In [ ]:
files_plugin = kernel.import_semantic_skill_from_directory(base_plugin , "FilePlugin")
answer_plugin = kernel.import_semantic_skill_from_directory(base_plugin , "AnswerPlugin")

In [ ]:
nodes_files = os.listdir("./data/notes")
transcrips_files = os.listdir("./data/transcripts")

In [ ]:
kblist = []

In [ ]:
for f in nodes_files:
    file = open("./data/notes/"+f, "r") 
    content = file.read()
    notesFunc = files_plugin["Notes"]
    result = notesFunc(content)
    mdcontent = result.result.replace("```json","").replace("```","")
    json_result = json.loads(mdcontent)
    kblist.append(json_result)
    file.close()
    break

In [ ]:
for f in transcrips_files:
    file = open("./data/transcripts/"+f, "r") 
    content = file.read()
    transcripsFunc = files_plugin["Transcrips"]
    result = transcripsFunc(content)
    txtcontent = result.result.replace("```json","").replace("```","")
    json_result = json.loads(txtcontent)
    for item in json_result:
        kblist.append(item)
    file.close()

In [ ]:
kblist

In [ ]:
base_vectordb = 'aboutMLKBDemoDemo'

In [ ]:

from semantic_kernel.connectors.memory.qdrant import QdrantMemoryStore

In [ ]:
qdrant_store = QdrantMemoryStore(vector_size=1536, url="http://localhost",port=6333)
await qdrant_store.create_collection_async(base_vectordb)

In [ ]:
kernel.register_memory_store(memory_store=qdrant_store)

In [ ]:
import uuid

In [ ]:
uuid.uuid4()

In [ ]:
for item in kblist:
    content = item["kb"] + ' - ' + item["content"]
    id =str(uuid.uuid4())
    await kernel.memory.save_information_async(base_vectordb, id=id, text=content)

In [ ]:
ask = "can you tell me what is different ML and AI"

memories = await kernel.memory.search_async(
    base_vectordb, ask, limit=1, min_relevance_score=0.8
)

In [ ]:
result = ''
for memory in memories:
    print(f"Top Result: {memory.text} with score {memory.relevance}")
    result = memory.text